00. Environment setup

In [78]:
%pip install google-cloud-bigquery
%pip install google-cloud
%pip install google-cloud-vision
%pip install tqdm
%pip install numpy
%pip install tensorflow
%pip install matplotlib
%pip install gensim

1398912.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


1398918.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


1398924.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


1398929.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


UnboundLocalError: local variable 'child' referenced before assignment

1398935.43s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


01. Connect to BigQuery

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('/users/fahimkhan/Downloads/endless-upgrade-415215-8ffb43b4399b.json')

project_id = 'endless-upgrade-415215'
client = bigquery.Client(credentials= credentials,project=project_id)

/Users/fahimkhan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


02. Query for Train and Test discharge summary datasets

In [ ]:
train_query = client.query("""
  SELECT TEXT FROM `endless-upgrade-415215.mimic3_notes.noteevents` 
  WHERE CATEGORY = "Discharge summary"
  LIMIT 50
 """)

test_query = client.query("""
  SELECT TEXT FROM `endless-upgrade-415215.mimic3_notes.noteevents` 
  WHERE CATEGORY = "Discharge summary"
  LIMIT 5
 """)

train_ds = train_query.result()
test_ds = test_query.result()

03. Process Discharge Summaries

In [ ]:
import os
import gensim

def read_corpus(results, tokens_only=False):
    i = 0
    for row in results:
        tokens = gensim.utils.simple_preprocess(row[0])
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
        i+=1

train_corpus = list(read_corpus(train_ds))
test_corpus = list(read_corpus(test_ds, tokens_only=True))

04. Training the Doc2Vector model

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

05. Assessing the model

In [83]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 50})


06. Show discharge summary doc vectors

In [87]:
first_dv = model.dv[0]
print("Vec Shape: ", first_dv.shape)
print("Doc Vec: ", first_dv)

Vec Shape:  (50,)
Doc Vec:  [-2.2312658  -2.215132   -2.6054485   2.5397565   1.6512975   2.1886888
  0.20984565 -1.1319172  -0.761361    1.5950024   0.39694232  0.1536958
 -1.1398039  -0.10939915  0.0193459  -1.9875625   1.0664362   1.2978393
 -1.8532673  -0.64533967  0.40191326  1.3622193  -0.27700597  1.5405405
  0.47022748  2.2191973  -4.0455256  -2.8532155  -3.077175   -1.7530426
  3.1924093   0.88553554 -1.6229337   0.7036333  -1.9573753   0.86098856
 -0.63737315 -1.5800264  -2.4868934   2.7872458   2.1566598  -1.2780693
 -2.6752417  -2.585372    0.7308647  -1.4411166  -0.5156833  -1.049187
  1.0807748  -1.4505929 ]
